In [1]:
import pandas as pd
import numpy as np
import requests
import json
import schedule
import datetime

# Create class to store/append data
class store_data:
    def __init__(self):
        self.data = pd.DataFrame()

    def update_data(self, start, end, order_sizes):
        # Get slippage in both directions
        new_row_1 = get_slippage(start, end, order_sizes)
        new_row_2 = get_slippage(end, start, order_sizes)
        # Append data, reset index, fill NAs
        self.data = pd.concat([self.data, new_row_1, new_row_2], axis = 0).reset_index(drop=True).fillna(0)
        self.data.to_csv('slippage.csv', mode='a', header=False, index=False)
        
def get_slippage(start, end, order_sizes):
    '''
    start = symbol to sell (str)
    end = symbol to buy (str)
    order_sizes = list of amounts to trade
    '''

    # Cycle through order sizes
    merged_results = [] 
    for order_size in order_sizes:
        # access dex api
        dex_URL = f"https://api-v2.dex.ag/price?from={start}&to={end}&fromAmount={order_size}&dex=ag"
        page = requests.get(dex_URL)
        dex = json.loads(page.text)
       
        results = {}
        # store entries
        results['dai_to_usdc'] = 1 if start == 'DAI' else 0
        results['amount'] = order_size
        results['price'] = dex['price']

        # add liquidity
        for item in dex['liquidity']:
            results['liquidity_' + item] = dex['liquidity'][item]

        merged_results.append(results)
    merged_results = pd.DataFrame(merged_results)

    # DF management
    # Fill NAs
    merged_results = merged_results.fillna(0)

    # Cast price as float, all other columns as int
    merged_results['price'] = merged_results['price'].astype(float)
    merged_results[merged_results.drop(['price'], axis =1).columns] = merged_results[merged_results.drop(['price'], axis =1).columns].astype(int)

    # Calc slippage
    for order_size in order_sizes:
        merged_results[str(order_size) + '_slippage'] = merged_results['price'] - merged_results[merged_results['amount'] == order_size]['price'].repeat(4).reset_index(drop=True)
        merged_results[str(order_size) + '_slippage'] = [y if y > 0 else 0 for y in merged_results[str(order_size) + '_slippage']]

    # timestamp
    merged_results['date'] = datetime.datetime.now().date()
    merged_results['time'] = datetime.datetime.now().time()

    return merged_results

# Instantiate store_data object
slippage = store_data()

In [2]:
# Run slippage pull every 30min
schedule.every(30).minutes.do(slippage.update_data, start='DAI',  end='USDC', order_sizes=[200000, 400000, 800000, 1600000])

while True:
    schedule.run_pending()



KeyboardInterrupt: 

In [106]:
slippage.update_data(start='DAI',  end='USDC', order_sizes=[200000, 400000, 800000, 1600000])
slippage.data

,1600000_slippage,200000_slippage,400000_slippage,800000_slippage,amount,dai_to_usdc,date,liquidity_curve_susd,liquidity_curvefi,liquidity_oasis,liquidity_uniswap_v2,liquidity_zero_x,price,time
0,0.001615,0,0.000288,0.000785,200000,1,2020-08-03,64,33,0,1.0,2.0,1.014535,23:36:13.038463
1,0.001328,0,0.000000,0.000497,400000,1,2020-08-03,59,39,0,1.0,1.0,1.014247,23:36:13.038463
2,0.000830,0,0.000000,0.000000,800000,1,2020-08-03,51,39,8,1.0,1.0,1.013750,23:36:13.038463
3,0.000000,0,0.000000,0.000000,1600000,1,2020-08-03,50,42,6,0.0,2.0,1.012920,23:36:13.038463
4,0.001994,0,0.000275,0.000839,200000,0,2020-08-03,43,57,0,NaN,NaN,0.984461,23:36:20.739378
5,0.001719,0,0.000000,0.000564,400000,0,2020-08-03,49,51,0,NaN,NaN,0.984186,23:36:20.739378
6,0.001155,0,0.000000,0.000000,800000,0,2020-08-03,52,48,0,NaN,NaN,0.983622,23:36:20.739378
7,0.000000,0,0.000000,0.000000,1600000,0,2020-08-03,52,44,4,NaN,NaN,0.982467,23:36:20.739378


,1600000_slippage,200000_slippage,400000_slippage,800000_slippage,amount,dai_to_usdc,date,liquidity_curve_susd,liquidity_curvefi,liquidity_oasis,liquidity_uniswap_v2,liquidity_zero_x,price,time
0,0.001563,0,0.000279,0.000767,200000,1,2020-08-04,53,45,0.0,NaN,2.0,1.014572,00:08:13.842727
1,0.001285,0,0.000000,0.000488,400000,1,2020-08-04,53,46,0.0,NaN,1.0,1.014293,00:08:13.842727
2,0.000797,0,0.000000,0.000000,800000,1,2020-08-04,48,41,7.0,NaN,4.0,1.013805,00:08:13.842727
3,0.000000,0,0.000000,0.000000,1600000,1,2020-08-04,48,43,6.0,NaN,3.0,1.013008,00:08:13.842727
4,0.001973,0,0.000268,0.000829,200000,0,2020-08-04,55,45,0.0,NaN,0.0,0.984384,00:08:18.986078
5,0.001705,0,0.000000,0.000561,400000,0,2020-08-04,55,45,0.0,NaN,0.0,0.984116,00:08:18.986078
6,0.001144,0,0.000000,0.000000,800000,0,2020-08-04,55,45,0.0,NaN,0.0,0.983555,00:08:18.986078
7,0.000000,0,0.000000,0.000000,1600000,0,2020-08-04,53,42,4.0,NaN,1.0,0.982411,00:08:18.986078
8,0.001556,0,0.000303,0.000765,200000,1,2020-08-04,30,68,0.0,NaN,2.0,1.014336,00:38:23.678386
9,0.001253,0,0.000000,0.000462,400000,1,2020-08-04,41,57,0.0,NaN,2.0,1.014033,00:38:23.678386
